Users can write the same qs in different form. so the qs is the style change in the prompt going to have a lot of effect in the output??

How LLM is behaving?

__consistency:__ Given a prompt LLM will generate multiple responses across different Query related to same qs and we will store in a list. In the below example it is 3(num_responses=3). Now across the list for one response check similarity for all other responses. Then take the mean of the similarity.
It should be high. When LLM generates two responses for two promts related to same qs you hope that two responses have very high cosine similarity..



__specificity:__ Very simple way to look at the LLM output. How many new words are there essentially. typically  it is 50%-70% new words will be there because always you will get some common words if you run it across different query.

__relevance:__ take embeddings of your expected response and llm output. Are they nearby across different Query? take cosine similarity at base level.

In [1]:
import os
import numpy as np
import pandas as pd
from langchain_groq import ChatGroq
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import getpass


In [2]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("enter key")

enter key ········


In [4]:
class PromptEval:
    def __init__(self):
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=1.2) 
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        
    def compute_specificity(self, response):
        words = response.split()
        unique_words = set(words)
        return len(unique_words) / len(words) if words else 0
    
    def compute_relevance(self, response, expected_content):
        embeddings = self.sentence_model.encode([response, expected_content])
        return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    def compute_consistency(self, prompt, num_responses=3):
        similarities = []
        responses = [self.llm.invoke(prompt).content for _ in range(num_responses)]
        for i in range(len(responses)):
            for j in range(i+1, len(responses)):
                similarities.append(self.compute_relevance(responses[i], responses[j]))
        return np.mean(similarities)

    def compare_prompts(self, prompts, expected_content):
        results = []
        for prompt in prompts:
            response = self.llm.invoke(prompt).content
            relevance = self.compute_relevance(response, expected_content)
            specificity = self.compute_specificity(response)
            consistency = self.compute_consistency(prompt)
            results.append(
                {
                    # "response": response,
                    "relevance": relevance,
                    "specificity": specificity,
                    "consistency": consistency,
                }
            )
        results = pd.DataFrame.from_dict(results)
        return results


In [7]:
if __name__ == "__main__":
    eval = PromptEval()

    joey_speech = """It's a love based on giving and receiving, as well as having and sharing. And the love that they give and have is shared and received. And through this having and giving and sharing and receiving, we too can share and love and have... and receive.
    When we share the giving and experience the receiving, we find ourselves having what was received and receiving what was shared. For in sharing what we have and having what we share, the giving becomes receiving and the receiving becomes giving.
    The beauty of their bond comes from receiving what is given and giving what is had. As they share in having and have in sharing, we witness how receiving the shared and sharing the received creates a circle of having what is given and giving what is had.
    Life together means giving to sharing and sharing in receiving. When they receive what is shared and share what is received, they discover having given and giving had. Through receiving the having and having the receiving, sharing gives and giving shares.
    Their commitment shows how having received and receiving had leads to sharing given and giving shared. As they give to having and have in giving, the sharing receives while the receiving shares. This exchange of having shared and sharing had completes the giving received and receiving given
    """

    quantum_mechanics = """ Quantum mechanics reveals a universe fundamentally different from our everyday experience. At the subatomic level, particles exhibit wave-like properties, existing in multiple states simultaneously until measured. This principle, known as superposition, challenges our classical intuition.
    Heisenberg's uncertainty principle demonstrates we cannot precisely know both position and momentum of a particle. Quantum entanglement allows particles to maintain instantaneous connections regardless of distance, what Einstein called "spooky action at a distance."
    Quantum tunneling enables particles to pass through seemingly impenetrable barriers. Wave function collapse occurs when observation forces quantum systems to resolve into definite states.
    These counterintuitive phenomena form the backbone of modern technology—from lasers to MRI machines to transistors in computing devices. While mathematically robust, quantum theory continues to spark philosophical debates about reality's true nature and the role of consciousness in the universe.
    Understanding quantum mechanics requires abandoning classical determinism for probabilistic thinking, reminding us that the cosmos operates by rules far stranger and more beautiful than we initially imagined.RetryClaude can make mistakes. Please double-check responses.
    """

    joey_specificity = eval.compute_specificity(joey_speech)
    quantum_specificity = eval.compute_specificity(quantum_mechanics)

    prompts = [
        "List the types of machine learning.",
        "What are the main categories of machine learning algorithms?",
        "Explain the different approaches to machine learning.",
        "What types of machine learning can you think about? Explain like you are Shakespear, Darwin, or Einstein. Choose any of these personality"
    ]

    expected_content = "The main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning."
    compare_prompt_results = eval.compare_prompts(prompts, expected_content)



In [8]:
compare_prompt_results

,relevance,specificity,consistency
0,0.805258,0.630137,0.864563
1,0.661953,0.506297,0.829358
2,0.639991,0.480216,0.846719
3,0.431326,0.607908,0.805804


#Node all this numbers also have some variance so we need to do it maybe more times(here ist is 3 only)